In [1]:
%pylab inline

from __future__ import (division, print_function)

import os
import sys
import copy
import fnmatch
import warnings
import collections

import numpy as np
import scipy
try:
    from scipy.stats import scoreatpercentile
except:
    scoreatpercentile = False
from scipy.interpolate import interp1d
import cPickle as pickle

# Astropy
from astropy.io import fits
from astropy    import units as u
from astropy.stats import sigma_clip
from astropy.table import Table, Column
from astropy.utils.console import ProgressBar

# AstroML
from astroML.plotting import hist

# Matplotlib related
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.ticker import NullFormatter
from matplotlib.ticker import MaxNLocator
# Matplotlib default settings
rcdef = plt.rcParams.copy()
pylab.rcParams['figure.figsize'] = 12, 10
pylab.rcParams['xtick.major.size'] = 8.0
pylab.rcParams['xtick.major.width'] = 2.5
pylab.rcParams['xtick.minor.size'] = 4.0
pylab.rcParams['xtick.minor.width'] = 2.5
pylab.rcParams['ytick.major.size'] = 8.0
pylab.rcParams['ytick.major.width'] = 2.5
pylab.rcParams['ytick.minor.size'] = 4.0
pylab.rcParams['ytick.minor.width'] = 2.5

# Personal
import hscUtils as hUtil
import galSBP
import coaddCutoutGalfitSimple as gSimple 

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
def loadGalfitOutput(pklFile):     
    """
    Load a .pkl file for GALFIT model 
    Return the GalfitParser object 
    """
    if not os.path.isfile(pklFile): 
        raise Exception("XXX Can not find the .pkl file : %s") % pklFile 
    else: 
        return pickle.load(open(pklFile, 'rb'))

In [4]:
def get1SerSize(cat, pklDir, zStr='z', idStr='objid_dr12', pix=0.168, 
                chisqThr=4.0, nSerThr=8.0, extinctionStr='extinction_i'):
    
    galUse = np.empty(len(cat), dtype=bool)
    galUse[:] = False
    
    galRe1Ser = []
    galAr1Ser = []
    galMag1Ser = []
    galNs1Ser = []

    chi1Ser = []
    aic1Ser = []
    bic1Ser = []

    for ii, galaxy in enumerate(cat): 
                
        galId = galaxy[idStr]
        pattern1Ser = '*' + str(galId) + '*1ser.pkl'
        pattern2Ser = '*' + str(galId) + '*2ser.pkl'
        
        pkl1SerFound = findProfile(pattern1Ser, pklDir)
        pkl1SerFound = findProfile(pattern1Ser, pklDir)

        if len(pkl1SerFound) == 1:
            
            pkl1SerFile = pkl1SerFound[0]
            obj1Ser = loadGalfitOutput(pkl1SerFile)
            
            if type(obj1Ser) is not numpy.core.records.recarray:
            
                nser1Ser  = obj1Ser.component_1.n
                chisq1Ser = obj1Ser.reduced_chisq
            
                try: 
                    ai = galaxy[extinctionStr]
                except Exception:
                    ai = 0.05
            
                if (chisq1Ser <= chisqThr) and (nser1Ser < nSerThr):
                
                    scale = pixKpc(galaxy[zStr], show=False)
                    distmod = hUtil.cosmoDistMod(galaxy[zStr])
                
                    aic1, bic1, hq1 = gSimple.galfitAIC(obj1Ser)
                
                    chi1Ser.append(chisq1Ser)
                    aic1Ser.append(aic1)
                    bic1Ser.append(bic1)
        
                    galRe1Ser.append(obj1Ser.component_1.re * scale)
                    galAr1Ser.append(obj1Ser.component_1.ar)
                    galMag1Ser.append(obj1Ser.component_1.mag - ai - distmod)
                    galNs1Ser.append(obj1Ser.component_1.n)
                    
                    galUse[ii] = True
                    
    print("### Number of models : %d" % len(galRe1Ser))
    
    return galUse, galRe1Ser, galAr1Ser, galMag1Ser, galNs1Ser, chi1Ser, bic1Ser

In [5]:
def get2SerSize(cat, pklDir, zStr='z', idStr='objid_dr12', pix=0.168, 
                chisqThr=4.0, nSerThr=8.0, extinctionStr='extinction_i', 
                factor=0.95, massStr='logms_pca'):
    
    galUse = np.empty(len(cat), dtype=bool)
    galUse[:] = False
    
    galRe1Ser = []
    galAr1Ser = []
    galMag1Ser = []
    galNs1Ser = []

    chi1Ser = []
    aic1Ser = []
    bic1Ser = []
    
    galRe2SerI = []
    galAr2SerI = []
    galMag2SerI = []
    galNs2SerI = []
    
    galRe2SerO = []
    galAr2SerO = []
    galMag2SerO = []
    galNs2SerO = []
    
    innerFrac = []
    
    chi2Ser = []
    aic2Ser = []
    bic2Ser = []
    
    logmI = []
    logmO = []
    
    for ii, galaxy in enumerate(cat): 
                
        galId = galaxy[idStr]
        pattern1Ser = '*' + str(galId) + '*1ser.pkl'
        pattern2Ser = '*' + str(galId) + '*2ser.pkl'
        
        pkl1SerFound = findProfile(pattern1Ser, pklDir)
        pkl2SerFound = findProfile(pattern2Ser, pklDir)

        if (len(pkl1SerFound) == 1) and (len(pkl2SerFound) == 1):
            
            pkl1SerFile = pkl1SerFound[0]
            obj1Ser = loadGalfitOutput(pkl1SerFile)
            
            pkl2SerFile = pkl2SerFound[0]
            obj2Ser = loadGalfitOutput(pkl2SerFile)
            
            if (type(obj1Ser) is not numpy.core.records.recarray) and (
                type(obj2Ser) is not numpy.core.records.recarray):
            
                nser2A = obj1Ser.component_1.n
                nser2B = obj2Ser.component_2.n
                
                chisq1 = obj1Ser.reduced_chisq
                aic1, bic1, hq1 = gSimple.galfitAIC(obj1Ser)
                
                chisq2 = obj1Ser.reduced_chisq
                aic2, bic2, hq2 = gSimple.galfitAIC(obj2Ser)
            
                try: 
                    ai = galaxy[extinctionStr]
                except Exception:
                    ai = 0.05
            
                logmT = galaxy[massStr]

                if (logmT >= 11.2) and (bic2 < bic1) and (aic2 < aic1) and (
                    nser2A <= nSerThr) and (nser2B <= nSerThr):
                
                    scale = pixKpc(galaxy[zStr], show=False)
                    distmod = hUtil.cosmoDistMod(galaxy[zStr])
                
                    chi1Ser.append(chisq1)
                    aic1Ser.append(aic1)
                    bic1Ser.append(bic1)
        
                    galRe1Ser.append(obj1Ser.component_1.re * np.sqrt(obj1Ser.component_1.ar) * scale)
                    galAr1Ser.append(obj1Ser.component_1.ar)
                    galMag1Ser.append(obj1Ser.component_1.mag - ai - distmod)
                    galNs1Ser.append(obj1Ser.component_1.n)
                    
                    """ 2 Sersic part """
                    chi2Ser.append(chisq2)
                    aic2Ser.append(aic2)
                    bic2Ser.append(bic2)
                    
                    if (obj2Ser.component_1.re <= obj2Ser.component_2.re): 
                        compInner = obj2Ser.component_1
                        compOuter = obj2Ser.component_2
                    else: 
                        compInner = obj2Ser.component_2 
                        compOuter = obj2Ser.component_1

                    galRe2SerI.append(compInner.re * scale * np.sqrt(compInner.ar))
                    galAr2SerI.append(compInner.ar)
                    galMag2SerI.append(compInner.mag)
                    galNs2SerI.append(compInner.n)

                    galRe2SerO.append(compOuter.re * scale * np.sqrt(compOuter.ar))
                    galAr2SerO.append(compOuter.ar)
                    galMag2SerO.append(compOuter.mag)
                    galNs2SerO.append(compOuter.n)  
                                        
                    fluxI = 10.0 ** ((27.0 - compInner.mag) / 2.5)
                    fluxO = 10.0 ** ((27.0 - compOuter.mag) / 2.5)
                    innerFrac.append(fluxI / (fluxI + fluxO))
                    
                    logmI.append(logmT + np.log10(fluxI / (fluxI + fluxO)))
                    logmO.append(logmT + np.log10(fluxO / (fluxI + fluxO)))

                    galUse[ii] = True
                    
    print("### Number of models : %d" % len(galRe1Ser))
    
    return galRe2SerI, galRe2SerO, logmI, logmO